In [1]:
import numpy as np
import torch

## transformers

In [2]:
import models.u_transformers_2D as ut2d

In [3]:
w, h = 256,256
trans = ut2d.u_transformers_2D(filters = [4,8,16,32,64], trans_shape=(64,16,16), n_classes=2,in_channels=1)
x = torch.from_numpy(np.random.rand(2,1,w, h)).float()
y = trans(x)

In [1]:
32*16

512

In [2]:
(1024*((512/(2**4)))**2)/(1000**2)

1.048576

In [16]:
(512*(512/(2**3))**2)/(1000**2)

2.097152

In [18]:
4398046511104/(1000**3)

4398.046511104

--------------------

In [2]:
def softDice(pred, target, smoothing=1, nonSquared=False):
    intersection = (pred * target).sum(dim=(1, 2, 3))
    if nonSquared:
        union = (pred).sum() + (target).sum()
    else:
        union = (pred * pred).sum(dim=(1, 2, 3)) + (target * target).sum(dim=(1, 2, 3))
    dice = (2 * intersection + smoothing) / (union + smoothing)

    #fix nans
    dice[dice != dice] = dice.new_tensor([1.0])

    return dice.mean()

def dice(pred, target):
    predBin = (pred > 0.5).float()
    return softDice(predBin, target, 0, True).item()

In [4]:
y = torch.from_numpy(np.ones((1,1,80,80,32)))
p = torch.from_numpy(np.ones((1,1,80,80,32)))

In [5]:
dice(y,p)

0.03125

### Reversible

In [1]:
def count_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [2]:
import expconfigs.multi_atlas_revunet_01_v2 as cfg_rev
excfg_rev = cfg_rev.ExpConfig()
nr = (count_parameters(excfg_rev.net))
print(excfg_rev.experiment_name)

atlas_revunet_3D_5_v2


### Classic

In [3]:
import expconfigs.multi_atlas_unet_res01_v2 as cfg_cla
excfg_cla = cfg_cla.ExpConfig()
nc = (count_parameters(excfg_cla.net))

/home/myr/these/segmentation3D/models/networks_other.py:42: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight.data, a=0, mode='fan_in')


### Compare

In [6]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))

rev : 114557582
cla : 94122318
nc/nr : 0.8216157879449655
nr/nc : 1.2171139049082917
min : 0.8216157879449655


In [4]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))

rev : 76837774
cla : 94122318
nc/nr : 1.2249485259684905
nr/nc : 0.8163608337822704
min : 0.8163608337822704


In [4]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))

rev : 39117966
cla : 94122318
nc/nr : 2.406114827135951
nr/nc : 0.41560776265624905
min : 0.41560776265624905
